In [27]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense


from sklearn.preprocessing import MinMaxScaler

In [41]:
df = pd.read_csv(Path("../data/EMR.csv"))
df["Date"] = pd.DatetimeIndex(df["Date"]).date

df["Date"]=pd.to_datetime(df.Date,format="%Y-%m-%d")
# df.index=df['Date']

data=df.sort_index(ascending=True,axis=0)

# df.index = df["Date"]
# df = df.drop(axis=0, columns="Date")
data.head()

,Date,Close/Last,Volume,Open,High,Low
0,2021-06-25,$96,2076210,$95.76,$96.49,$95.47
1,2021-06-24,$95.35,2585059,$94.97,$95.42,$94.14
2,2021-06-23,$94.50,2234949,$95.11,$96.03,$94.45
3,2021-06-22,$94.78,2058333,$94.75,$95.52,$93.96
4,2021-06-21,$94.93,2580230,$93.22,$95.08,$93.01


In [42]:
new_dataset=pd.DataFrame(index=range(0,len(df)),columns=["Date","Close/Last"])

for i in range(0,len(data)):
    new_dataset["Date"][i]=data["Date"][i]
    new_dataset["Close/Last"][i]=data["Close/Last"][i]

new_dataset = new.replace({'\$':''}, regex = True)    

new_dataset

,Date,Close/Last
0,2021-06-25 00:00:00,$96
1,2021-06-24 00:00:00,$95.35
2,2021-06-23 00:00:00,$94.50
3,2021-06-22 00:00:00,$94.78
4,2021-06-21 00:00:00,$94.93
...,...,...
2512,2011-07-01 00:00:00,$57.09
2513,2011-06-30 00:00:00,$56.25
2514,2011-06-29 00:00:00,$54.93
2515,2011-06-28 00:00:00,$54.78


TypeError: float() argument must be a string or a number, not 'Timestamp'